## Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('Data/40 train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
###Drop Nan Values
df=df.dropna()


In [5]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.3.0'

In [12]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [18]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [19]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4672, 1611, 4696, 1504, 3195, 3195, 4313, 1017, 3173, 4372],
 [182, 4412, 3449, 2560, 1608, 1951, 2568],
 [2246, 3589, 3948, 2508],
 [827, 3769, 3631, 4052, 854, 2523],
 [516, 1608, 3958, 3779, 1960, 2266, 1608, 3030, 2138, 3345],
 [2927,
  983,
  287,
  3560,
  1921,
  3070,
  2293,
  2245,
  4313,
  1155,
  2797,
  4476,
  4951,
  278,
  2568],
 [4255, 305, 3882, 2111, 3340, 4332, 2049, 4112, 453, 2400, 433],
 [1005, 1452, 4683, 3061, 3727, 3483, 3070, 2359, 453, 2400, 433],
 [4778, 3820, 608, 4357, 4647, 4082, 2841, 2439, 3070, 4702],
 [1499, 4689, 2177, 3519, 4909, 4029, 1279, 772],
 [2945, 1524, 2781, 3228, 2397, 329, 1565, 4856, 2607, 1073, 1046],
 [4052, 3009, 3195, 4082, 3070, 3727],
 [2681, 4052, 1007, 1164, 2976, 3661, 2467, 1361, 2103],
 [2216, 2615, 2704, 1143, 1642, 2896, 2220, 453, 2400, 433],
 [2046, 1633, 2137, 4470, 3289, 453, 2400, 433],
 [1444, 2517, 2097, 4588, 49, 1777, 3207, 1504, 422, 4313],
 [220, 2823, 4412],
 [924, 2138, 2904, 2276, 3070, 2445, 1963, 2568],


### Embedding Representation

In [20]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1017 3173 4372]
 [   0    0    0 ... 1608 1951 2568]
 [   0    0    0 ... 3589 3948 2508]
 ...
 [   0    0    0 ...  453 2400  433]
 [   0    0    0 ... 3136  617 4686]
 [   0    0    0 ... 2925 2399 2603]]


In [21]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4672,
       1611, 4696, 1504, 3195, 3195, 4313, 1017, 3173, 4372])

In [22]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs),y.shape

(18285, (18285,))

In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [28]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 53ms/step - loss: 0.3028 - accuracy: 0.8556 - val_loss: 0.2000 - val_accuracy: 0.9148
Epoch 2/10
192/192 [==============================] - 8s 42ms/step - loss: 0.1367 - accuracy: 0.9474 - val_loss: 0.1997 - val_accuracy: 0.9198
Epoch 3/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0976 - accuracy: 0.9635 - val_loss: 0.2043 - val_accuracy: 0.9200
Epoch 4/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0733 - accuracy: 0.9740 - val_loss: 0.2517 - val_accuracy: 0.9128
Epoch 5/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0528 - accuracy: 0.9819 - val_loss: 0.2823 - val_accuracy: 0.9170
Epoch 6/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0324 - accuracy: 0.9909 - val_loss: 0.3670 - val_accuracy: 0.9143
Epoch 7/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0220 - accuracy: 0.9931 - val_loss: 0.4005 - val_accuracy: 0.912

### Performance Metrics And Accuracy

In [29]:

y_pred1=model1.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [30]:
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(y_test,y_pred1)

array([[3131,  288],
       [ 255, 2361]], dtype=int64)

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9100248550124275

In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.89      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

